Running TEM simulation

In [4]:
"""
This file runs a training simulation for the Whittington et al. 2020 agent, the Tolman-Eichenbaum Machine (TEM).
The TEM is a model of the hippocampus that learns to navigate a series of environments and solve a series of tasks.
"""

import os

import numpy as np

from neuralplayground.agents.whittington_2020 import Whittington2020 
## imports the TEM model ("agent")
from neuralplayground.agents.whittington_2020_extras import whittington_2020_parameters as parameters 
## provides all the parameters/settings for the model
from neuralplayground.arenas import Simple2D #changed 
## defines area/world. batch env: , simple2D:
from neuralplayground.backend import SingleSim, tem_training_loop
## simulation manager and specific training procedure for model/agent
from neuralplayground.experiments import Sargolini2006Data
## import experimental data (not really used in this code)

simulation_id = "TEM_arena_sim"
save_path = os.path.join(os.getcwd(), "results_sim")
# save_path = os.path.join(os.getcwd(), "examples", "agent_examples", "trained_results")
agent_class = Whittington2020
env_class = Simple2D
training_loop = tem_training_loop
## defines the core components of the simulation/model


arena_x_limits = (0, 7)
arena_y_limits = (0, 7)

#room_widths = [int(arena_x_limits[0][1] - arena_x_limits[0][0])]
#room_depths = [int(arena_y_limits[0][1] - arena_y_limits[0][0])]

#calculating width and depth for a single arena
room_widths = [arena_x_limits[1] - arena_x_limits[0]]
room_depths = [arena_y_limits[1] - arena_y_limits[0]]

#room_widths = [int(np.diff(arena_x_limits)[i]) for i in range(len(arena_x_limits))]
#room_depths = [int(np.diff(arena_y_limits)[i]) for i in range(len(arena_y_limits))]

#discrete_env_params = {
    #"environment_name": "DiscreteObject",
    #"state_density": 1,
    #"n_objects": params["n_x"],
    #"agent_step_size": 1,
    #"use_behavioural_data": False,
    #"data_path": None,
    #"experiment_class": Sargolini2006Data,
#}

env_params = {
    "environment_name": "Simple2D_1",
    "arena_x_limits": arena_x_limits,
    "arena_y_limits": arena_y_limits,
}

params = parameters.parameters()
full_agent_params = params.copy()
full_agent_params["n_x"] = room_widths[0] * room_depths[0]
full_agent_params["batch_size"] = 1
   

agent_params = {
    "model_name": "Whittington2020",
    "params": full_agent_params,
    "batch_size": 1,
    "room_widths": room_widths,
    "room_depths": room_depths,   
    "state_densities": [1], #not sure 
    "use_behavioural_data": False,
}


# Full model training consists of 20000 episodes
training_loop_params = {"n_episode": 10, "params": full_agent_params}

sim = SingleSim(
    simulation_id=simulation_id,
    agent_class=agent_class,
    agent_params=agent_params,
    env_class=env_class,
    env_params=env_params,
    training_loop=training_loop,
    training_loop_params=training_loop_params,
)

# print(sim)
print("Running sim...")
sim.run_sim(save_path)
print("Sim finished.")


IndexError: invalid index to scalar variable.

Plotting TEM Results:

In [ ]:
SystemExit

In [ ]:
import importlib
import os
import pickle

import pandas as pd

from neuralplayground.comparison import GridScorer
from neuralplayground.plotting import PlotSim

In [ ]:
simulation_id = "TEM_custom_plot_sim"
from neuralplayground.saved_models import fetch_model_path

#save_path = fetch_model_path("whittington_2020_in_discritized_objects")
print(save_path)
plotting_loop_params = {"n_episode": 5000}

In [ ]:
training_dict = pd.read_pickle(os.path.join(os.getcwd(), save_path, "params.dict"))
model_weights = pd.read_pickle(os.path.join(save_path, "agent"))
model_spec = importlib.util.spec_from_file_location("model", save_path + "/whittington_2020_model.py")
model = importlib.util.module_from_spec(model_spec)
model_spec.loader.exec_module(model)
params = pd.read_pickle(os.path.join(save_path, "agent_hyper"))
tem = model.Model(params)
tem.load_state_dict(model_weights)
tem.eval()

In [ ]:
sim = PlotSim(
    simulation_id=simulation_id,
    agent_class=training_dict["agent_class"],
    agent_params=training_dict["agent_params"],
    env_class=training_dict["env_class"],
    env_params=training_dict["env_params"],
    plotting_loop_params=plotting_loop_params,
)

In [ ]:
trained_agent, trained_env = sim.plot_sim(save_path, n_walks=1000, random_state=False, custom_state=[0.5,0.5])
trained_env.plot_trajectories();

In [ ]:
with open(os.path.join(save_path, "NPG_environments.pkl"), "rb") as f:
    environments = pickle.load(f)
with open(os.path.join(save_path, "NPG_model_input.pkl"), "rb") as f:
    model_input = pickle.load(f)

In [ ]:
training_dict["params"] = training_dict["agent_params"]
del training_dict["agent_params"]
agent = training_dict["agent_class"](**training_dict["params"])
agent.plot_run(tem, model_input, environments)

In [ ]:
agent.plot_rate_map(rate_map_type='g')
agent.plot_rate_map(rate_map_type='p');

In [ ]:
rate_map_mat = agent.get_rate_map_matrix(agent.g_rates,1,2)
GridScorer = GridScorer(rate_map_mat.shape[0])
GridScorer.plot_grid_score(r_out_im = rate_map_mat, plot= True)
score = GridScorer.get_scores(rate_map_mat)